In [71]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

# Parameters Config

In [72]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [73]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+ '

RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [74]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [75]:
ctable = CharacterTable(chars)

In [76]:
ctable.indices_char
#ctable.char_indices
#ctable.chars

{0: ' ',
 1: '+',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

# Data Generation

In [77]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [78]:
print(questions[:5], expected[:5])

['0+7    ', '5+581  ', '844+0  ', '1+60   ', '182+391'] ['7   ', '586 ', '844 ', '61  ', '573 ']


# Processing

In [79]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [80]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Data:
(60000, 7, 12)
(60000, 4, 12)


In [81]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False False False  True False
   False]
  [False False False False False False False False False False  True
   False]
  [False False False False  True False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False  True False False False False False False False
   False]
  [False False False False False False False False False False False
    True]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False False False  True
   False]
  [False False False False False False False False False False  True
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False False  True False
   False]
  [False False False False  True False False False False False False
   False]
  [False False  True False False False Fal

# Build Model

In [82]:
print('Build model...')

model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
    
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_5 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [83]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 8s 458us/step - loss: 2.0226 - acc: 0.2897 - val_loss: 1.8570 - val_acc: 0.3269
Q 75+896  T 971  ☒ 110 
Q 312+231 T 543  ☒ 113 
Q 455+73  T 528  ☒ 113 
Q 589+914 T 1503 ☒ 1101
Q 484+648 T 1132 ☒ 110 
Q 27+103  T 130  ☒ 133 
Q 24+375  T 399  ☒ 113 
Q 313+56  T 369  ☒ 133 
Q 22+588  T 610  ☒ 113 
Q 245+40  T 285  ☒ 113 

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 318us/step - loss: 1.8490 - acc: 0.3314 - val_loss: 1.8492 - val_acc: 0.3235
Q 25+269  T 294  ☒ 122 
Q 989+18  T 1007 ☒ 1098
Q 222+83  T 305  ☒ 129 
Q 71+54   T 125  ☒ 129 
Q 465+141 T 606  ☒ 112 
Q 61+141  T 202  ☒ 122 
Q 23+823  T 846  ☒ 122 
Q 939+14  T 953  ☒ 109 
Q 158+88  T 246  ☒ 109 
Q 731+675 T 1406 ☒ 112 

-------------------

18000/18000 [==============================] - 6s 310us/step - loss: 1.2144 - acc: 0.5489 - val_loss: 1.2108 - val_acc: 0.5447
Q 903+1   T 904  ☒ 913 
Q 972+12  T 984  ☒ 987 
Q 831+130 T 961  ☒ 906 
Q 11+231  T 242  ☒ 134 
Q 89+11   T 100  ☒ 119 
Q 711+8   T 719  ☑ 719 
Q 9+350   T 359  ☒ 313 
Q 13+579  T 592  ☒ 593 
Q 97+267  T 364  ☒ 356 
Q 454+338 T 792  ☒ 879 

--------------------------------------------------
Iteration 15
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 317us/step - loss: 1.1767 - acc: 0.5635 - val_loss: 1.1798 - val_acc: 0.5609
Q 36+925  T 961  ☒ 968 
Q 302+378 T 680  ☒ 686 
Q 67+348  T 415  ☒ 434 
Q 649+68  T 717  ☒ 745 
Q 823+32  T 855  ☑ 855 
Q 87+615  T 702  ☒ 794 
Q 534+318 T 852  ☒ 876 
Q 489+439 T 928  ☒ 900 
Q 65+52   T 117  ☒ 107 
Q 634+96  T 730  ☒ 732 

--------------------------------------------------
Iteration 16
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 6s 316us/step - loss: 0.7618 - acc: 0.7143 - val_loss: 0.8063 - val_acc: 0.6859
Q 245+567 T 812  ☒ 813 
Q 651+977 T 1628 ☒ 1531
Q 84+789  T 873  ☒ 879 
Q 93+48   T 141  ☒ 132 
Q 269+29  T 298  ☒ 307 
Q 283+427 T 710  ☒ 666 
Q 644+878 T 1522 ☒ 1521
Q 73+717  T 790  ☑ 790 
Q 55+774  T 829  ☑ 829 
Q 45+213  T 258  ☑ 258 

--------------------------------------------------
Iteration 29
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 322us/step - loss: 0.7206 - acc: 0.7302 - val_loss: 0.7552 - val_acc: 0.7037
Q 77+24   T 101  ☑ 101 
Q 227+35  T 262  ☒ 271 
Q 370+43  T 413  ☒ 414 
Q 24+375  T 399  ☒ 390 
Q 67+903  T 970  ☒ 978 
Q 95+630  T 725  ☒ 716 
Q 85+487  T 572  ☒ 573 
Q 544+808 T 1352 ☒ 1348
Q 403+374 T 777  ☒ 766 
Q 830+636 T 1466 ☒ 1366

--------------------------------------------------
Iteration 30
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 6s 328us/step - loss: 0.2604 - acc: 0.9321 - val_loss: 0.3173 - val_acc: 0.8960
Q 90+682  T 772  ☑ 772 
Q 550+66  T 616  ☒ 617 
Q 313+33  T 346  ☒ 356 
Q 10+989  T 999  ☒ 9009
Q 75+210  T 285  ☒ 275 
Q 225+48  T 273  ☑ 273 
Q 64+556  T 620  ☑ 620 
Q 265+21  T 286  ☑ 286 
Q 913+973 T 1886 ☒ 1877
Q 457+48  T 505  ☑ 505 

--------------------------------------------------
Iteration 43
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 320us/step - loss: 0.2372 - acc: 0.9412 - val_loss: 0.3322 - val_acc: 0.8887
Q 43+12   T 55   ☑ 55  
Q 304+544 T 848  ☒ 858 
Q 64+598  T 662  ☑ 662 
Q 901+82  T 983  ☑ 983 
Q 7+768   T 775  ☑ 775 
Q 634+24  T 658  ☑ 658 
Q 829+947 T 1776 ☒ 1786
Q 34+508  T 542  ☑ 542 
Q 114+775 T 889  ☒ 899 
Q 35+126  T 161  ☑ 161 

--------------------------------------------------
Iteration 44
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 6s 319us/step - loss: 0.0927 - acc: 0.9831 - val_loss: 0.1696 - val_acc: 0.9440
Q 999+12  T 1011 ☑ 1011
Q 575+198 T 773  ☑ 773 
Q 591+489 T 1080 ☑ 1080
Q 72+880  T 952  ☑ 952 
Q 82+305  T 387  ☑ 387 
Q 71+350  T 421  ☑ 421 
Q 214+19  T 233  ☑ 233 
Q 778+986 T 1764 ☑ 1764
Q 988+10  T 998  ☑ 998 
Q 310+75  T 385  ☑ 385 

--------------------------------------------------
Iteration 57
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 320us/step - loss: 0.0832 - acc: 0.9868 - val_loss: 0.1642 - val_acc: 0.9462
Q 404+93  T 497  ☑ 497 
Q 963+76  T 1039 ☒ 1049
Q 249+5   T 254  ☑ 254 
Q 131+549 T 680  ☑ 680 
Q 4+449   T 453  ☑ 453 
Q 578+68  T 646  ☑ 646 
Q 21+766  T 787  ☑ 787 
Q 940+19  T 959  ☑ 959 
Q 62+608  T 670  ☑ 670 
Q 417+31  T 448  ☑ 448 

--------------------------------------------------
Iteration 58
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 6s 328us/step - loss: 0.0355 - acc: 0.9964 - val_loss: 0.1161 - val_acc: 0.9610
Q 848+37  T 885  ☑ 885 
Q 700+9   T 709  ☑ 709 
Q 4+463   T 467  ☑ 467 
Q 981+7   T 988  ☑ 988 
Q 872+920 T 1792 ☑ 1792
Q 911+908 T 1819 ☑ 1819
Q 28+243  T 271  ☑ 271 
Q 917+922 T 1839 ☒ 1849
Q 44+850  T 894  ☑ 894 
Q 3+981   T 984  ☑ 984 

--------------------------------------------------
Iteration 71
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 321us/step - loss: 0.0333 - acc: 0.9966 - val_loss: 0.1196 - val_acc: 0.9577
Q 523+892 T 1415 ☑ 1415
Q 620+252 T 872  ☑ 872 
Q 58+157  T 215  ☑ 215 
Q 82+695  T 777  ☑ 777 
Q 597+417 T 1014 ☑ 1014
Q 62+697  T 759  ☒ 758 
Q 13+158  T 171  ☑ 171 
Q 364+965 T 1329 ☒ 1339
Q 506+917 T 1423 ☑ 1423
Q 833+501 T 1334 ☒ 1344

--------------------------------------------------
Iteration 72
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 6s 325us/step - loss: 0.0385 - acc: 0.9911 - val_loss: 0.1030 - val_acc: 0.9637
Q 291+912 T 1203 ☑ 1203
Q 240+49  T 289  ☑ 289 
Q 487+9   T 496  ☒ 497 
Q 44+356  T 400  ☒ 300 
Q 375+7   T 382  ☑ 382 
Q 298+173 T 471  ☑ 471 
Q 427+457 T 884  ☑ 884 
Q 591+489 T 1080 ☑ 1080
Q 305+513 T 818  ☑ 818 
Q 61+407  T 468  ☑ 468 

--------------------------------------------------
Iteration 85
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 322us/step - loss: 0.0171 - acc: 0.9990 - val_loss: 0.0907 - val_acc: 0.9696
Q 359+667 T 1026 ☑ 1026
Q 89+11   T 100  ☒ 90  
Q 108+872 T 980  ☒ 989 
Q 268+39  T 307  ☑ 307 
Q 334+90  T 424  ☑ 424 
Q 201+798 T 999  ☒ 909 
Q 99+727  T 826  ☑ 826 
Q 944+51  T 995  ☒ 195 
Q 185+28  T 213  ☑ 213 
Q 971+258 T 1229 ☑ 1229

--------------------------------------------------
Iteration 86
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 6s 319us/step - loss: 0.0095 - acc: 0.9998 - val_loss: 0.0835 - val_acc: 0.9729
Q 411+819 T 1230 ☑ 1230
Q 173+81  T 254  ☑ 254 
Q 791+842 T 1633 ☑ 1633
Q 90+140  T 230  ☒ 220 
Q 607+28  T 635  ☑ 635 
Q 960+756 T 1716 ☑ 1716
Q 244+6   T 250  ☑ 250 
Q 720+25  T 745  ☑ 745 
Q 80+606  T 686  ☑ 686 
Q 49+471  T 520  ☑ 520 

--------------------------------------------------
Iteration 99
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 6s 329us/step - loss: 0.0078 - acc: 0.9999 - val_loss: 0.0815 - val_acc: 0.9721
Q 750+0   T 750  ☑ 750 
Q 187+830 T 1017 ☑ 1017
Q 31+72   T 103  ☑ 103 
Q 548+605 T 1153 ☑ 1153
Q 91+255  T 346  ☑ 346 
Q 114+775 T 889  ☑ 889 
Q 471+391 T 862  ☑ 862 
Q 597+417 T 1014 ☑ 1014
Q 133+400 T 533  ☒ 543 
Q 626+144 T 770  ☑ 770 


# Testing

In [84]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 
#####################################################
equalAnswer = 0
predictions = model.predict_classes(test_x)

for i in range(len(test_x)):
    predAnswer = ctable.decode(predictions[i],False)
    correctAnswer = ctable.decode(test_y[i])
    if(predAnswer == correctAnswer):
        equalAnswer+=1        

accuracyAnswer = equalAnswer/len(test_y)
accuracyAnswer

MSG : Prediction


0.9033